# 合成資料生成模組 - python3.6 版本

## 環境建置
1. 安裝 pyenv 依賴
```
sudo apt update
sudo apt install -y make build-essential libssl-dev zlib1g-dev \
  libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm \
  libncursesw5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev liblzma-dev
```
2. 安裝 Python 3.6.15
```
pyenv install 3.6.15
pyenv local 3.6.15
```
3. 建立 virtualenv
```
python -m venv venv
source venv/bin/activate
```
4. 安裝 core 套件（預先裝 wheel）
```
pip install --upgrade pip setuptools wheel
pip install numpy==1.19.5 cython==0.29.36 pandas==0.24.2
pip install sdv==0.3.6
pip install jupyterlab==3.2.9
pip install scipy==1.2.3 sdmetrics==0.0.2.dev0
```

In [1]:
import argparse
import json
import logging
import os
import time

import numpy as np
import pandas as pd
from sdv.tabular import GaussianCopula
from sdv.tabular import CTGAN

from sdv import SDV

In [14]:
def get_data_from_model(model_path, num_rows=1000):
    logging.info("Generate Synthetic data from Model")
    model = SDV.load(model_path)
    
    sampled = model.sample(num_rows=num_rows)
    
    return sampled

In [15]:
def data_sythesizer(args, input_df=pd.DataFrame()):
    """Synthesize input dataframe data and output dataframe

    Args:
        args (argparse): arguments for configs.
        input_df (DataFrame): input data. Defaults to pd.DataFrame().

    Returns:
        output_df (DataFrame): synthesized data output
    """
    pri_key = args.primary_key

    if args.synth_model == "GaussianCopula":
        logging.info("sythetic model arch: GaussianCopula")
        model = GaussianCopula(primary_key=pri_key) if pri_key else GaussianCopula()
    elif args.synth_model == "CTGAN":
        if args.custom_setting:
            logging.info("sythetic model arch: CTGAN-c")
            model = CTGAN(
                primary_key=pri_key,
                epochs=args.epochs,
                batch_size=args.batch_size,
                generator_dim=tuple(args.gen_dim),
                discriminator_dim=tuple(args.dis_dim),
                verbose=True,
            )
        else:
            logging.info("sythetic model arch: CTGAN")
            model = (
                CTGAN(primary_key=pri_key, verbose=True)
                if pri_key
                else CTGAN(verbose=True)
            )
    else:
        logging.info("the sythetic model is not supported!")

    logging.info("Synthetic model fitting data start ... ")
    start_time = time.time()
    model.fit(input_df)
    logging.info(f"Training time cost: {time.time()-start_time}")
    output_df = model.sample(num_rows=args.num_rows)

    if args.save_model:
        output_model_path = os.path.join(
            args.output_dir, f"syn_model_{args.synth_model}.pkl"
        )
        if args.custom_setting:
            output_model_path = os.path.join(
                args.output_dir, f"syn_model_{args.synth_model}-c.pkl"
            )

        logging.info(f"Save model to {output_model_path}")
        model.save(output_model_path)

    return output_df

In [16]:
def set_args(args_list=None):
    """Main Function
    process input and do configs check
    """
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--input_syn_model", type=str, default=None, help="path to your syn_data model file")
    parser.add_argument("--output_fpath", type=str, default=None, help="set full file path for your syn_data output csv")
    parser.add_argument("--synth_model", type=str, default="GaussianCopula", help="sythetic model type")
    parser.add_argument("--primary_key", type=str, default="", help="primary key in your tabular data")
    parser.add_argument("--num_rows", type=int, default=200, help="num rows of the output sythetic dataframe")
    parser.add_argument("--save_model", action="store_true", help="set for save model pkl file")
    parser.add_argument("--save_output", action="store_true", help="set for save output csv file")
    parser.add_argument("--save_report", action="store_true", help="set for save report csv and image files")
    
    parser.add_argument("--custom_setting", action="store_true", help="set for custom setting in CTGAN and TVAE Model")
    parser.add_argument("--epochs", type=int, default=300, help="set epochs for training CTGAN and TVAE Model")
    parser.add_argument("--batch_size", type=int, default=500, help="set batch size for training CTGAN and TVAE Model")
    parser.add_argument("--gen_dim", type=int, nargs="+", default=[256, 256], help="set gen dimension")
    parser.add_argument("--dis_dim", type=int, nargs="+", default=[256, 256], help="set dis dimension")
    

    return parser.parse_args(args_list)

In [21]:
def main(args):
    # logging.info(f"contents of args.primary_key {args.primary_key}")
    # logging.info(f"contents of args.custom_setting {args.custom_setting}")
    # logging.info(f"contents of args.gen_dim {args.gen_dim}")
    # logging.info(f"contents of args.dis_dim {args.dis_dim}")
    if not args.input_syn_model:
        assert os.path.exists(args.input_path), f"Can't find the input file at {args.input_path}."
        assert os.path.exists(args.output_dir), f"Can't find the output folder at {args.output_dir}."
        assert args.synth_model in ["GaussianCopula", "CTGAN"]
        # ["GaussianCopula", "CTGAN", "CopulaGAN", "TVAE"]

        input_path = args.input_path
        output_dir = args.output_dir
        # output_fname=args.output_fname

        input_df = pd.read_csv(input_path)
        if args.primary_key:
            assert args.primary_key in input_df.columns

        # if "Id" in input_df.columns:
        #     input_df = input_df.drop(columns=["Id"])

        output_df = data_sythesizer(args=args, input_df=input_df)

        logging.info("output dataframe shape")
        logging.info(output_df.shape)
        logging.info("output dataframe head(5)")
        logging.info(output_df.head())

        if args.save_output:
            base = os.path.basename(input_path)
            output_fname = (
                os.path.splitext(base)[0] + "_" + args.synth_model + "_output.csv"
            )
            output_df.to_csv(os.path.join(output_dir, output_fname), index=False)
    else:
        model_path = args.input_syn_model
        num_rows = args.num_rows
        output_fpath=args.output_fpath
        assert os.path.exists(model_path), f"Can't find the model_path pkl file: {model_path}."
        output_df = get_data_from_model(model_path, num_rows=num_rows)
        
        logging.info("output dataframe shape")
        logging.info(output_df.shape)
        logging.info("output dataframe head(5)")
        logging.info(output_df.head())

        if args.save_output:
            output_df.to_csv(output_fpath, index=False)


# 使用模組訓練合成資料模型 + 生成合成資料

In [22]:
args = set_args([
    "--input_syn_model", "output/syn_model_GaussianCopula.pkl", # 合成資料生成模型路徑 
    "--output_fpath", "output/syn_data.csv",   # 合成資料輸出路徑
    "--num_rows", "100",  # 生成的資料筆數
    "--save_output"
])

main(args)

## 檢視真實資料與合成資料表單

In [23]:
import pandas as pd

real_data_df = pd.read_csv("input/data.csv")  # 真實資料路徑
syn_data_df = pd.read_csv("output/syn_data.csv")  # 合成資料預設檔名為: 真實資料檔名 + "_GaussianCopula_output"

In [24]:
real_data_df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1118,20,RL,57.0,9764,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2008,WD,Normal,130000
1,903,60,RL,63.0,7875,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,WD,Normal,180000
2,658,70,RL,60.0,7200,Pave,NaN,Reg,HLS,AllPub,...,0,NaN,MnPrv,NaN,0,2,2008,WD,Normal,149000
3,339,20,RL,91.0,14145,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,400,5,2006,WD,Normal,202500
4,341,60,RL,85.0,14191,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,202900
5,553,20,RL,87.0,11146,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,255500
6,1122,20,RL,84.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2006,New,Partial,212900
7,1116,20,RL,93.0,12085,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,New,Partial,318000
8,1433,30,RL,60.0,10800,Pave,Grvl,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,64500
9,489,190,RL,60.0,10800,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,ConLD,Normal,160000


In [25]:
syn_data_df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,26,RL,90.601113,12153,Pave,NaN,IR1,Lvl,AllPub,...,9,NaN,NaN,NaN,-61,6,2010,WD,Normal,170502
1,1,208,RL,NaN,2158,Pave,NaN,IR1,Lvl,AllPub,...,-1,NaN,NaN,Shed,272,4,2008,WD,Normal,246971
2,2,181,FV,40.166015,13743,Pave,NaN,IR2,Lvl,AllPub,...,-8,NaN,NaN,NaN,40,8,2008,New,Normal,234705
3,3,19,RL,82.133128,20994,Pave,NaN,Reg,Lvl,AllPub,...,5,NaN,GdPrv,Shed,177,4,2009,WD,Normal,352735
4,4,5,RL,81.948840,12669,Pave,NaN,Reg,Lvl,AllPub,...,6,NaN,NaN,NaN,117,7,2010,WD,Normal,123210
5,5,30,RL,60.043795,11461,Pave,NaN,Reg,Lvl,AllPub,...,7,NaN,NaN,NaN,-31,10,2006,WD,Normal,236723
6,6,22,RL,68.093432,10964,Pave,NaN,IR1,Lvl,AllPub,...,6,NaN,NaN,NaN,-151,1,2009,WD,Normal,136951
7,7,56,RL,NaN,11767,Pave,NaN,IR1,HLS,AllPub,...,2,NaN,NaN,NaN,58,5,2008,WD,Partial,169397
8,8,56,RL,NaN,20629,Pave,NaN,IR1,Lvl,AllPub,...,11,NaN,NaN,NaN,-74,4,2006,New,Partial,186430
9,9,20,RL,81.467539,13848,Pave,NaN,IR1,Low,AllPub,...,-11,NaN,NaN,NaN,105,6,2007,WD,Normal,181048
